======================================

In [134]:
# !/usr/bin/env python3
import requests
import time
from datetime import date
from datetime import datetime


def formattime(h, m, s):
    if len(h) == 1:
        hour = '0' + h
    else:
        hour = h
    if len(m) == 1:
        minute = '0' + m
    else:
        minute = m
    if len(s) == 1:
        second = '0' + s
    else:
        second = s
    return hour, minute, second


def get_observer_location():
    """Returns the longitude and latitude for the location of this machine.
    Returns:str: latitude str: longitude"""
    url = 'http://ip-api.com/json/'
    payload = {}
    headers = {}
    response = requests.request("GET", url, headers=headers, data=payload)
    data = response.json()

    print('From ' + data['city'] + ' ' + data['region'] + ' at ' + time.strftime('%l:%M%p %Z on %b %d, %Y'))
    return data['lat'], data['lon']


def get_sun_position(latitude, longitude):
    ASTRONOMYAPI_ID = "11179991-9c28-41d6-a6a5-828a307caec9"
    ASTRONOMYAPI_SECRET = "173d393e195729e5f3e114e1c6d77e3fe4084720a6581cddc51dd8c4943da30847462bd2175744e1ced438b49161ac34791dbc56c4f71810246f52ad4cadbb116614ddd86af9c46021ebacbb71ebb2bd35c46da1b9015f2baf3d6d72687ab47cc69b3785fb376bdf5cec5dcb6579b351"

    """Returns the current position of the sun in the sky at the specified location
    Parameters:
    latitude (str)
    longitude (str)

    Returns:
    float: azimuth
    float: altitude
    "https://api.astronomyapi.com/api/v2/bodies/positions/?latitude=47.5079&longitude=-122.177&elevation=2&from_date=2022-05-23&to_date=2022-05-25&time=08:00:00"
    """
    today = date.today()
    now = datetime.now()
    hour, min, sec = formattime(str(now.hour), str(now.minute), str(now.second))
    time = hour + ':' + min + ':' + sec
    url = 'https://api.astronomyapi.com/api/v2/bodies/positions?latitude=' + str(latitude) + '&longitude=' + str(longitude) + '&elevation=2&from_date=' + str(today) + '&to_date=' + str(today) + '&time=' + time

    payload = {}
    headers = {'Authorization': 'Basic: ZDI1ODA1MzYtNDcxMC00NmFjLTlhZTktZjJhYzgzMDQ2NmQ1OjcwYzg3NDYxMzYwZDczOTFkNzU3OTdhNTY1NmViNWFkOTg4MDEwNTRhY2NiZTgzZWRhYTcwYjFlMDhmN2EyNTcyMzY5NTdiNjI4MGExODNmODhiZGJiZWM0OTQ2ODBjM2U1MjA3N2RjYzQyMmUyZDRlOWVlMDczZDVhMmRjNDk3NTExYjE0MzgzOTljNDE0NDAxOThkNDNhODU3MmZkYTFlN2M1YWFkMzI4Y2JmZmJkZmM1YWU3MjUzMWRlNmFiMjA3N2Q0MzdiM2ZkNTYzNDE1MmUwODkxNTkzYjExNGQy'}
    response = requests.request("GET", url, headers=headers, data=payload)

    if response.status_code != 200:
        print('No connection ********* STATUS CODE: ', response.status_code, ' ', response.reason)

    data = response.json()
    # print(data.keys())
    for record in data['data']['table']['rows']:
        # print(record['cells'][0]['position'])
        if record['cells'][0]['id'] == 'sun':
            # pprint(record['cells'][0]['position']['horizonal']['azimuth'])
            azimuth = record['cells'][0]['position']['horizonal']['azimuth']['degrees']
            altitude = record['cells'][0]['position']['horizonal']['altitude']['degrees']
            distance = record['cells'][0]['distance']['fromEarth']['km']
            print('Sun: Distance from Earth: ' + distance + ' km')
    return azimuth, altitude


def print_position(azimuth, altitude):
    """Prints the position of the sun in the sky using the supplied coordinates
    Parameters:
    azimuth (float)
    altitude (float)
    """
    print("The Sun is currently at: " + azimuth + ' degrees azimuth ' + altitude + ' degrees altitude')


if __name__ == "__main__":
    latitude, longitude = get_observer_location()
    azimuth, altitude = get_sun_position(latitude, longitude)
    print_position(azimuth, altitude)


From Denver CO at  6:53PM PDT on May 25, 2022
Sun: Distance from Earth: 151533925.32176 km
The Sun is currently at: 285.98 degrees azimuth 14.05 degrees altitude


(base) kb@Lenovo:~/gitrepo/iea-cohort-07/06_rest/astronomy-lab$ python3 solar_test3.py -f list.txt
From Denver CO at  5:36PM PDT on May 24, 2022
Sun: Distance from Earth: 151502940.87587 km
The Sun is currently at: 274.51 degrees azimuth 28.38 degrees altitude

(base) kb@Lenovo:~/gitrepo/iea-cohort-07/06_rest/astronomy-lab$ python3 solar_test3.py -f list.txt
From Renton WA at  6:43PM PDT on May 24, 2022
Sun: Distance from Earth: 151506388.09227 km
The Sun is currently at: 280.58 degrees azimuth 19.02 degrees altitude


=================================

#!/usr/bin/env python3

import requests
import argparse
from time import sleep
from tabulate import tabulate
from operator import itemgetter


ASTRONOMYAPI_ID = "11179991-9c28-41d6-a6a5-828a307caec9"
ASTRONOMYAPI_SECRET = "173d393e195729e5f3e114e1c6d77e3fe4084720a6581cddc51dd8c4943da30847462bd2175744e1ced438b49161ac34791dbc56c4f71810246f52ad4cadbb116614ddd86af9c46021ebacbb71ebb2bd35c46da1b9015f2baf3d6d72687ab47cc69b3785fb376bdf5cec5dcb6579b351"


def get_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('-i', '--ip-address', dest='ip', help='IP address to lookup')
    parser.add_argument('-f', '--file', dest='file', help='List of IP addresses to lookup')

    options = parser.parse_args()
    return options


options = get_args()
located_list = []
def locate(obj):
    ip = options.ip
    url = f'http://ip-api.com/json/{options.ip}?fields=query,lat,lon,isp'

    response = requests.get(url)
    resp_data = response.json()
    ip_data = {'ip_address': ip}
    latitude = resp_data['lat']
    longitude = resp_data['lon']
    results = {**ip_data, **resp_data}
    located_list.append(results)


    return resp_data[latitude], resp_data[longitude]
    sleep(2)


def process_file(ip_file):
    ip_list = []

    with open(options.file, 'r') as f:
        lines = f.readlines()

        for line in lines:
            ip_list.append(line.strip())
        for ip in ip_list:
            options.ip = ip
            locate(options.ip)

if options.ip:
    locate(options.ip)
elif options.file:
    process_file(options.file)

print(tabulate(sorted(located_list, key=itemgetter('ip_address', 'query', 'lat', 'lon', 'isp')), headers='keys'))


    